# 데이터 불러오기

In [30]:
# from google.colab import drive
# drive.mount('/content/drive')

- 시드값

In [1]:
SEED = 42

- 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/"
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [52]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.2_피처삭제X_군집분석_1108.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.2_피처삭제X_군집분석_1108.csv")

train_ft.shape , test_ft.shape

((14940, 1471), (12225, 1471))

In [65]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv")

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

# 결측치 처리

In [66]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [67]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

- 추가 피처 만들어 보기

In [36]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

## Feature Encoding

In [68]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    246
dtype: int64

In [69]:
import category_encoders as ce

enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1507), (12225, 1507))

In [70]:
import category_encoders as ce

enc = ce.count.CountEncoder()
# train_ft["주구매지점_cnt"] = enc.fit_transform(train_ft[["주구매지점"]])
# test_ft["주구매지점_cnt"] = enc.transform(test_ft[["주구매지점"]])

train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])
train_ft.shape, test_ft.shape

((14940, 1509), (12225, 1509))

- 문자열 피처 삭제

In [71]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [72]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1506), (12225, 1506))

In [73]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [74]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

# 정답 데이터

In [75]:
target = train_target["target"]

# cv 점수 확인해보기

# 임계값 조정

In [15]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier

# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):  # 콜백함수 역할
        hp = {  "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 5)  }
        model = XGBClassifier(**hp, random_state= self.seed)    
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials= 500) 

print("Best trial:")
print(study.best_trial.params)

[I 2024-11-11 00:44:06,325] A new study created in memory with name: no-name-c7f03b32-9673-4955-b248-61ce53d18bac
[I 2024-11-11 00:45:32,808] Trial 0 finished with value: 0.696949724507624 and parameters: {'n_estimators': 400, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.696949724507624.
[I 2024-11-11 00:47:42,017] Trial 1 finished with value: 0.7138924584313022 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.7138924584313022.
[I 2024-11-11 00:48:24,128] Trial 2 finished with value: 0.7044579517701782 and parameters: {'n_estimators': 200, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample': 

KeyboardInterrupt: 

In [ ]:
study.best_trial.params #0.7252440006884955

{'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


# v3.0 : 0.7211477612229629
# v3.2_피처삭제X_군집분석 : 0.7216530444546538
# 

Mean F1 Macro Score: 0.7216530444546538


In [76]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

pred_proba = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred_proba.append(model.predict_proba(x_valid))
    

In [77]:
pred = np.mean(
        [   pred_proba[0],
            pred_proba[1],
            pred_proba[2],
            pred_proba[3],
            pred_proba[4],
            
            ],axis=0)
pred.shape

(2988, 2)

In [78]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# 모델 예측 확률값 (예: predict_proba 결과)
proba = pred[:, 1]  # 양성 클래스의 확률값
y_true = y_valid  # 실제 타겟값

# 임계값 범위 설정
thresholds = np.arange(0.1, 0.9, 0.01)

# 각 임계값에서의 성능 저장
results = []

for threshold in thresholds:
    # 확률값 기반으로 예측 생성
    y_pred = (proba >= threshold).astype(int)
    
    # 성능 지표 계산 (F1, Precision, Recall)
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    results.append((threshold, f1, precision, recall))

# 최적 임계값 찾기 (F1 기준)
best_threshold, best_f1, _, _ = max(results, key=lambda x: x[1])

# 결과 출력
print(f"최적 임계값: {best_threshold:.2f}")
print(f"F1 Score (최적): {best_f1:.4f}")

# 결과를 데이터프레임으로 정리 (선택사항)
import pandas as pd
pd.set_option('display.max_rows', None)
results_df = pd.DataFrame(results, columns=['Threshold', 'F1', 'Precision', 'Recall'])
results_df

최적 임계값: 0.41
F1 Score (최적): 0.6008


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

,Threshold,F1,Precision,Recall
0,0.10,0.288499,0.393950,0.998296
1,0.11,0.290155,0.394143,0.997445
2,0.12,0.293806,0.394941,0.997445
3,0.13,0.299846,0.396277,0.997445
4,0.14,0.302662,0.396745,0.996593
5,0.15,0.307839,0.397754,0.995741
6,0.16,0.314142,0.399043,0.994889
7,0.17,0.320761,0.399794,0.990630
8,0.18,0.325344,0.399931,0.985520
9,0.19,0.333791,0.401045,0.980409


In [79]:
params = {'n_estimators': 1000,
 'learning_rate': 0.027075406407767497,
 'max_depth': 5,
 'min_child_weight': 7,
 'subsample': 0.6789816859997232,
 'colsample_bytree': 0.6682531834544282,
 'gamma': 1.5046881781916308}

model = XGBClassifier(**params)
model.fit(train_ft, target)
pred = model.predict(test_ft)

In [81]:
pred = model.predict_proba(test_ft)[:,1]
submit["target"] = pred
submit.to_csv("v3.0_XGB_cluster_proba.csv",index=False)

In [82]:
pred = model.predict_proba(test_ft)[:,1]
pred = (pred >= 0.41).astype(int)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [83]:
submit["target"] = pred
submit.to_csv("v3.0_XGB_cluster_0.41.csv",index=False)